In [1]:
# GerryChain is a library for using Markov Chain Monte Carlo methods to study the problem of political redistricting. 
#   Development of the library began during the 2018 Voting Rights Data Institute (VRDI).
#
# Install instructions: https://gerrychain.readthedocs.io/en/latest/user/install.html

from gerrychain import Graph

# Read Iowa county graph from the json file "IA_county.json"
filepath = 'C:\\Users\\antay\\Downloads\\IA\\'
filename = 'IA_county.json'

# GerryChain has a built-in function for reading graphs of this type:
G = Graph.from_json( filepath + filename )

In [2]:
# For each node, print the node #, county name, and its population
for node in G.nodes:
    county_name = G.nodes[node]['NAME20']
    county_population = G.nodes[node]['P0010001']
    G.nodes[node]['TOTPOP'] = county_population
    print("Node",node,"represents",county_name,"County, which had a population of",county_population,"in the 2020 census.")

Node 0 represents Wright County, which had a population of 12943 in the 2020 census.
Node 1 represents Montgomery County, which had a population of 10330 in the 2020 census.
Node 2 represents Union County, which had a population of 12138 in the 2020 census.
Node 3 represents Keokuk County, which had a population of 10033 in the 2020 census.
Node 4 represents Story County, which had a population of 98537 in the 2020 census.
Node 5 represents Mitchell County, which had a population of 10565 in the 2020 census.
Node 6 represents Grundy County, which had a population of 12329 in the 2020 census.
Node 7 represents Winneshiek County, which had a population of 20070 in the 2020 census.
Node 8 represents Polk County, which had a population of 492401 in the 2020 census.
Node 9 represents Sac County, which had a population of 9814 in the 2020 census.
Node 10 represents Marshall County, which had a population of 40105 in the 2020 census.
Node 11 represents Delaware County, which had a population 

In [3]:
# we are to solve the following task:
# input: a population vector, desired number of districts k
# output: a partition of the populations into k districts (not necessarily connected!) 
#            to minimize the difference between most and least populated districts

import gurobipy as gp
from gurobipy import GRB

In [4]:
k = 4  # desired number of districts

# create model 
m = gp.Model()

# create variables
x = m.addVars( G.nodes, k, vtype=GRB.BINARY )  # x[i,j] = 1 when county i is assigned to district j
y = m.addVar()                                 # y = population of smallest district
z = m.addVar()                                 # z = population of largest district

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-14


In [5]:
# objective is to minimize absolute population deviation
m.setObjective( z - y, GRB.MINIMIZE )

# add constraints saying that each county i is assigned to one district
m.addConstrs( gp.quicksum( x[i,j] for j in range(k) ) == 1 for i in G.nodes ) 

# add constraints saying that each district has population at least y
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes ) >= y for j in range(k) )

# add constraints saying that each district has population at most z
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes ) <= z for j in range(k) )

m.update()

In [6]:
# solve IP model
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 107 rows, 398 columns and 1196 nonzeros
Model fingerprint: 0x36c28af2
Variable types: 2 continuous, 396 integer (396 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 682278.00000
Presolve time: 0.00s
Presolved: 107 rows, 398 columns, 1196 nonzeros
Variable types: 0 continuous, 398 integer (396 binary)

Root relaxation: objective -1.164153e-10, 137 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.00000    0    8 682278.000   -0.00000  

In [7]:
# print the absolute population deviation
print("The minimum required deviation is",m.objVal,"persons.")

# retrieve the districts and their populations
districts = [ [ i for i in G.nodes if x[i,j].x > 0.5 ] for j in range(k) ]
district_county_names = [ [ G.nodes[i]['NAME20'] for i in district ] for district in districts ]
district_populations = [ sum( G.nodes[i]['TOTPOP'] for i in district ) for district in districts ]

# print district info
for j in range(k):
    print("District",j,"has these nodes =",districts[j],"and this population =",district_populations[j] )
    print("The corresponding county names are =",district_county_names[j] )
    print("")

The minimum required deviation is 1.0 persons.
District 0 has these nodes = [0, 1, 3, 17, 25, 27, 29, 35, 40, 41, 45, 54, 57, 65, 70, 81, 86, 88, 94, 96, 98] and this population = 797592
The corresponding county names are = ['Wright', 'Montgomery', 'Keokuk', 'Boone', 'Scott', 'Fremont', 'Butler', 'Osceola', 'Lyon', 'Adams', 'Tama', 'Washington', 'Winnebago', 'Harrison', 'Palo Alto', 'Hancock', 'Wayne', 'Linn', 'Marion', 'Appanoose', 'Johnson']

District 1 has these nodes = [4, 8, 12, 31, 56, 59, 78, 80] and this population = 797592
The corresponding county names are = ['Story', 'Polk', 'Davis', 'Woodbury', 'Allamakee', 'Cass', 'Warren', 'Chickasaw']

District 2 has these nodes = [2, 5, 6, 7, 15, 19, 20, 22, 23, 36, 42, 46, 49, 52, 53, 58, 61, 64, 74, 75, 76, 79, 82, 84, 85, 92, 93, 95, 97] and this population = 797592
The corresponding county names are = ['Union', 'Mitchell', 'Grundy', 'Winneshiek', 'Jones', 'Plymouth', 'Pottawattamie', 'Page', 'Cherokee', 'Cerro Gordo', 'Monona', 'Mon

In [8]:
import networkx as nx
for district in districts:
    print("Is district =", district, "connected?", nx.is_connected( G.subgraph( district ) ) )

Is district = [0, 1, 3, 17, 25, 27, 29, 35, 40, 41, 45, 54, 57, 65, 70, 81, 86, 88, 94, 96, 98] connected? False
Is district = [4, 8, 12, 31, 56, 59, 78, 80] connected? False
Is district = [2, 5, 6, 7, 15, 19, 20, 22, 23, 36, 42, 46, 49, 52, 53, 58, 61, 64, 74, 75, 76, 79, 82, 84, 85, 92, 93, 95, 97] connected? False
Is district = [9, 10, 11, 13, 14, 16, 18, 21, 24, 26, 28, 30, 32, 33, 34, 37, 38, 39, 43, 44, 47, 48, 50, 51, 55, 60, 62, 63, 66, 67, 68, 69, 71, 72, 73, 77, 83, 87, 89, 90, 91] connected? False
